In [ ]:
import pandas as pd
import numpy as np
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_pacf, plot_acf
import matplotlib.pyplot as plt

Section 1: Data Preprocessing

In [ ]:
# Read the dataset
df = pd.read_excel("../../data/market_data.xlsx") 

In [ ]:
# Remove variables that are not in the range of study
vic = df.filter(items=['Time (UTC+10)', 'Regions VIC Trading Price ($/MWh)',
                       'Regions VIC Operational Demand (MW)',
                       'Regions VIC Trading Total Intermittent Generation (MW)'])

vic = vic.rename(columns={'Time (UTC+10)': 'Time', 'Regions VIC Trading Price ($/MWh)': 'Spot Price',
                          'Regions VIC Operational Demand (MW)': 'Demand',
                          'Regions VIC Trading Total Intermittent Generation (MW)': 'Supply'})

Section 2: Non-transformed (Original) Time Series

In [ ]:
# attribute to Jason,B (2020), How to Check if Time Series Data is Stationary with Python, Machine Learning Mastery,
# https://machinelearningmastery.com/time-series-data-stationary-python/

def stationarity(data):
    '''Compute the mean and variance of two time series subsamples.'''
    
    split = len(data) / 2
    X1, X2 = data.loc[0:split, 'Spot Price'], data.loc[split:, 'Spot Price']
    
    mean1, mean2 = X1.mean(), X2.mean()
    var1, var2 = X1.var(), X2.var()

    print('mean_1=%f, mean_2=%f' % (mean1, mean2))
    print('variance_1=%f, variance_2=%f' % (var1, var2))
    
    return

In [ ]:
# attribute to Vijay,K (2021), Statistical tests to check stationarity in Time Series – Part 1, Analytics Vidhya,
# https://www.analyticsvidhya.com/blog/2021/06/statistical-tests-to-check-stationarity-in-time-series-part-1/

def adf_test(timeseries):
    '''Perform time series stationarity test.'''
    
    print ('Results of Dickey-Fuller Test:')
    
    dftest = adfuller(timeseries, autolag='AIC')
    dfoutput = pd.Series(dftest[0:4], index=['Test Statistic', 'p-value', '#Lags Used', 'Number of Observations Used'])
    
    for key,value in dftest[4].items():
        dfoutput['Critical Value (%s)'%key] = value
        
    print (dfoutput)
    
    return

In [ ]:
# Check stationarity in mean and variance of spot price time series
stationarity(vic)

In [ ]:
# Check stationarity in mean of spot price time series using a formal test
adf_test(vic['Spot Price'])

Section 3: Log-transformed Time Series

In [ ]:
vic_tf = vic.copy()

In [ ]:
# Perform log transformation on the attributes of interest
min_price = min(vic_tf['Spot Price'])
log_const = 1 - min_price

vic_tf['Spot Price'] = np.log(log_const + vic_tf['Spot Price'])
vic_tf['Demand'] = np.log(log_const + vic_tf['Demand'])
vic_tf['Supply'] = np.log(log_const + vic_tf['Supply'])

In [ ]:
# Check stationarity in mean and variance of spot price time series after log transformation
stationarity(vic_tf)

In [ ]:
# Check stationarity in mean of spot price time series after log transformation using a formal test
adf_test(vic_tf['Spot Price'])

In [ ]:
# Generate the partial autocorrelation plot
plot_pacf(vic_tf['Spot Price'], lags=10)
plt.show()

In [ ]:
# Generate the autocorrelation plot
plot_acf(vic_tf['Spot Price'])
plt.show()